# Google Kubernetes Engine Guestbook Tutorial

by: Coleman Word

https://console.cloud.google.com




>[Google Kubernetes Engine Guestbook Tutorial](#updateTitle=true&folderId=1RmTu1U3Lw-zFE3Q8gU5mWTFP3Q95z77t&scrollTo=uoCzVwNfUMQ9)

>>[Introduction](#updateTitle=true&folderId=1RmTu1U3Lw-zFE3Q8gU5mWTFP3Q95z77t&scrollTo=ezXd22gKcd2Q)

>>[Configure gcloud](#updateTitle=true&folderId=1RmTu1U3Lw-zFE3Q8gU5mWTFP3Q95z77t&scrollTo=FsbH57RsZDoE)

>>[Clone Repository](#updateTitle=true&folderId=1RmTu1U3Lw-zFE3Q8gU5mWTFP3Q95z77t&scrollTo=wzpPwsAXU8rW)

>>[Configuring your deployment](#updateTitle=true&folderId=1RmTu1U3Lw-zFE3Q8gU5mWTFP3Q95z77t&scrollTo=FbMMfKhVVcWQ)

>>>[Kind](#updateTitle=true&folderId=1RmTu1U3Lw-zFE3Q8gU5mWTFP3Q95z77t&scrollTo=_pGdfTeOVj6M)

>>>[Metadata](#updateTitle=true&folderId=1RmTu1U3Lw-zFE3Q8gU5mWTFP3Q95z77t&scrollTo=kQvN77fUVrWY)

>>>[Spec](#updateTitle=true&folderId=1RmTu1U3Lw-zFE3Q8gU5mWTFP3Q95z77t&scrollTo=ouhhbUtNVuZy)

>>[Exploring Service](#updateTitle=true&folderId=1RmTu1U3Lw-zFE3Q8gU5mWTFP3Q95z77t&scrollTo=cZkCmZ3qV7-q)

>>[Deploy Redis Master](#updateTitle=true&folderId=1RmTu1U3Lw-zFE3Q8gU5mWTFP3Q95z77t&scrollTo=ZdpT5zroWRGg)

>>>[Setup Credentials](#updateTitle=true&folderId=1RmTu1U3Lw-zFE3Q8gU5mWTFP3Q95z77t&scrollTo=CUXEg8QrWY5D)

>>>[Create a Service](#updateTitle=true&folderId=1RmTu1U3Lw-zFE3Q8gU5mWTFP3Q95z77t&scrollTo=T6XKF_7lWUaG)

>>>[Create a Replication Controller](#updateTitle=true&folderId=1RmTu1U3Lw-zFE3Q8gU5mWTFP3Q95z77t&scrollTo=xWn8k-eqWj5i)

>>>[Inspect Your Cluster](#updateTitle=true&folderId=1RmTu1U3Lw-zFE3Q8gU5mWTFP3Q95z77t&scrollTo=F5nZWR0gWpQ3)

>>>[Deploy the Redis Read Replicas](#updateTitle=true&folderId=1RmTu1U3Lw-zFE3Q8gU5mWTFP3Q95z77t&scrollTo=vzffxBtjbZmi)

>>>[Deploy the Guestbook Frontend](#updateTitle=true&folderId=1RmTu1U3Lw-zFE3Q8gU5mWTFP3Q95z77t&scrollTo=Uo4IbliQb0Dn)

>>>[Find External IP Address](#updateTitle=true&folderId=1RmTu1U3Lw-zFE3Q8gU5mWTFP3Q95z77t&scrollTo=cqSArtumcAiP)

>>>[Visit your running Guestbook app](#updateTitle=true&folderId=1RmTu1U3Lw-zFE3Q8gU5mWTFP3Q95z77t&scrollTo=MMHZXaRtcI2s)



## Introduction


This tutorial shows you how to create a Google Kubernetes Engine  cluster, and deploy a simple Guestbook application to the cluster. The multi-tier application you will deploy consists of a Redis  master, multiple Redis read replicas, and load-balanced web frontends.

## Configure gcloud

In [0]:
gcloud init --console-only

## Clone Repository

In [0]:
TUTORIALDIR=src/hardkubes/gke_guestbook-2018-06-19-17-10

In [0]:
git clone https://github.com/GoogleCloudPlatform/kubernetes-engine-samples.git $TUTORIALDIR

In [0]:
cd $TUTORIALDIR/guestbook

## Configuring your deployment


In [0]:
cat redis-master-controller.yaml

This file contains configuration to deploy a Redis master.



### Kind

Although we have a single instance of our redis master, we are using a Replication Controller to enforce that exactly one pod keeps running. E.g., if the node were to go down, the replication controller will ensure that the redis master gets restarted on a healthy node. (In our simplified example, this could result in data loss.)

### Metadata

The metadata: sections define labels to apply to the Replication Controller and related Pods. Labels are simple key-value pairs, which can be queried by other operations.

### Spec


Here we define the Pod specification which the Replication Controller will use to create the Redis pod. The image: tag refers to a Docker image to be pulled from a registry.

## Exploring Service

In [0]:
cat redis-master-service.yaml

This file contains configuration to deploy a Service for the Redis master. A Kubernetes service is a named load balancer that proxies traffic to one or more containers. This is done using the labels metadata that we defined in the redis-master pod. As mentioned, we have only one Redis master, but we nevertheless want to create a service for it. Why? Because it gives us a deterministic way to route to the single master using an elastic IP.

Services find the pods to load balance based on the pods' labels. The selector: field of the service description determines which pods will receive the traffic sent to the service, and the port: and targetPort: information defines what port the service proxy will run at.

## Deploy Redis Master

### Setup Credentials

In [0]:
gcloud container clusters get-credentials cluster-1 --zone us-central1-a

### Create a Service

In [0]:
kubectl create -f redis-master-service.yaml

### Create a Replication Controller

In [0]:
kubectl create -f redis-master-controller.yaml

### Inspect Your Cluster

In [0]:
kubectl get pods

In [0]:
kubectl get rc

In [0]:
kubectl get services

### Deploy the Redis Read Replicas

The Redis read replicas contain similar configuration in the guestbook/ directory. This time, we will use a single .yaml file to deploy both the Service and Replication Controller. The Replication Controller configuration contains replicas: 2 which will create 2 pods.

In [0]:
kubectl get pods

In [0]:
kubectl get rc

In [0]:
kubectl get services

### Deploy the Guestbook Frontend


A frontend pod is a simple PHP server that is configured to talk to either the slave or master services, depending on whether the client request is a read or a write. It exposes a simple AJAX interface, and serves an Angular-based UX. Again we'll create a set of replicated frontend pods instantiated by a replication controller— this time, with three replicas.

As with the other pods, we now want to create a service to group the frontend pods. The RC and service are described in the file frontend.yaml.



In [0]:
kubectl create -f all-in-one/frontend.yaml

### Find External IP Address

In [0]:
kubectl get services --watch

### Visit your running Guestbook app


Copy the IP in the External IP column.

Open a new tab and visit your Guestbook app, by navigating to the IP.